# WGMS reference glaciers - standard projections plots 
### based on _10 minutes to..._ a glacier change projection with GCM data

This notebook creates animations of the modeled evolution of certain glaciers from the year 2000 to the year 2001. The model is forced with the climate predictions of one member of the ISIMIP3b climate model for 3 different climate scenarios(ssp126, ssp370, ssp585).  
To visualize the animations the [oggm-3dviz library](https://github.com/OGGM/oggm-3dviz) is used.

# hi-res attempt

In [ ]:
from oggm import cfg, utils, workflow, tasks, DEFAULT_BASE_URL
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import os
import xarray as xr
from oggm.shop import gcm_climate
from os.path import join

In [ ]:
# we always need to initialzie and define a working directory
cfg.initialize(logging_level='WARNING')
cfg.PATHS['working_dir'] = '/Users/afisc/atmo_master/rgi_job/oggm-3dviz/wgms_hi-res'



In [ ]:
# glaciers chosen for visualization
glac_dict = {
    # Alaska
    "Wolverine Glacier": {
        "rgi_id": "RGI60-01.09162",
        "camera_params": {
            'azimuth': -135, # rotate around the z-axis
            'elevation': 20,  # elevation angle above ground
        },
        "fl_thickness_threshold": 1,
        "rolling_mean_smoothing": 5,
        "crop": {
            "x_crop": 0.8,
            "y_crop": 0.8
        },
    }, # check_geom: WARN:WasInvalid;
    # Arctic Canada North
    "White Glacier": {
        "rgi_id": "RGI60-03.04539",
        "camera_params": {
            'azimuth': -135, # rotate around the z-axis
                    'elevation': 20,  # elevation angle above ground
            },
        "fl_thickness_threshold": 1,
        "rolling_mean_smoothing": 5,
        "crop": {
            "x_crop": 0.65,
            "y_crop": 0.7
        },
        },
    # Svalbard
    "Austre Broeggerbreen": {
        "rgi_id": "RGI60-07.00504",
        "camera_params": {
            'azimuth': 45, # rotate around the z-axis
                    'elevation': 20,  # elevation angle above ground
            },
        "fl_thickness_threshold": 1,
        "rolling_mean_smoothing": 5
        },
    # Scandinavia
    "Nigardsbreen": {
        "rgi_id": "RGI60-08.01126",
        "camera_params": {
            'azimuth': -90, # rotate around the z-axis
                    'elevation': 20,  # elevation angle above ground
            },
        "fl_thickness_threshold": 1,
        "rolling_mean_smoothing": 5,
        "crop": {
            "x_crop": 0.8,
            "y_crop": 0.8
            }
        },
    # Central Europe
    "Pasterze": {
        "rgi_id": "RGI60-11.00106",
        "camera_params": {
            'azimuth': -90, # rotate around the z-axis
                    'elevation': 20,  # elevation angle above ground
            },
        "fl_thickness_threshold": 1.5, # had a little bit of flickering in the end of the ssp370 scenario
        "rolling_mean_smoothing": 5,
        "crop": {
            "x_crop": 0.8,
            "y_crop": 0.8
            }
        },
    # Caucasus
    "Garabashi Glacier": {
        "rgi_id": "RGI60-12.00161",
        "camera_params": {
            'azimuth': -135, # rotate around the z-axis
                    'elevation': 20,  # elevation angle above ground
            },
        "fl_thickness_threshold": 1,
        "rolling_mean_smoothing": 5,
        "crop": {
            "x_crop": 0.8,
            "y_crop": 0.8
            }
        },
    # Central Asia
    "Golubin Glacier": {
        "rgi_id": "RGI60-13.11609",
        "camera_params": {
            'azimuth': 75, # rotate around the z-axis
                    'elevation': 20,  # elevation angle above ground
            },
        "fl_thickness_threshold": 1,
        "rolling_mean_smoothing": 5,
        "crop": {
            "x_crop": 0.7,
            "y_crop": 0.7
            },
        },
    # Southern Andes
    "Echaurren Norte": {
        "rgi_id":"RGI60-17.13715",
        "camera_params": {
            'azimuth': -100, # rotate around the z-axis
                    'elevation': 20,  # elevation angle above ground
            },
        "fl_thickness_threshold": 3, # had a lot of flickering in (almost) all scenarios
        "rolling_mean_smoothing": 8,
        "crop": {
            "x_crop": 0.5,
            "y_crop": 0.5
            },
        },

    
    # additional Glaiciers
    # Southern Andes
    "Upsala + Cono": {
        "rgi_id":"RGI60-17.00172",
        "camera_params": {
            'azimuth': -135, # rotate around the z-axis
                    'elevation': 20,  # elevation angle above ground
            },
        "fl_thickness_threshold": 1, # had a lot of flickering in (almost) all scenarios
        "rolling_mean_smoothing": 5,
        "crop": {
            "x_crop": None,
            "y_crop": 0.85
            },
        },
    # Himalaya
    "Khumbu Glacier": {
        "rgi_id": "RGI60-15.03733",
        "camera_params": {
            'azimuth': 135, # rotate around the z-axis
                    'elevation': 20,  # elevation angle above ground
            },
        "fl_thickness_threshold": 1, # had a lot of flickering in (almost) all scenarios
        "rolling_mean_smoothing": 5,
        
        "crop": {
            "x_crop": 0.85,
            "y_crop": 0.85
            },
        },
    }

glac_names = list(glac_dict.keys())
rgi_ids = [glacier["rgi_id"] for glacier in glac_dict.values()]



In [ ]:
# Instruction for beginning with existing OGGM's preprocessed directories
load_from_prepro_base_url = True
if load_from_prepro_base_url:
    # to start from level 0 you can do
    prepro_base_url_L0 = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L1-L2_files/elev_bands/'
    gdirs = workflow.init_glacier_directories(rgi_ids,
                                              from_prepro_level=0,
                                              prepro_base_url=prepro_base_url_L0,
                                              prepro_border=80,  # could be 10, 80, 160 or 240
                                              reset=True,
                                              force=True,
                                             )

SET "dx"(grid size) HERE:

In [ ]:
# define the border, we keep the default here
cfg.PARAMS['border'] = cfg.PARAMS['border']

# set the method for determining the local grid resolution
cfg.PARAMS['grid_dx_method'] = 'fixed'  # The default method is 'square', which determines the grid spacing (dx) based on the glacier's outline area.
cfg.PARAMS['fixed_dx'] = 30  # This allows setting a specific resolution in meters. It's applicable only when grid_dx_method is set to 'fixed'.

# set the DEM source to use
source = 'COPDEM30'  # we stick with the OGGM default

# this task adds the DEM and defines the local grid
workflow.execute_entity_task(tasks.define_glacier_region, gdirs,
                             source=source);


In [ ]:
flowline_type_to_use = 'elevation_band'  # you can also select 'centerline' here

if flowline_type_to_use == 'elevation_band':
    elevation_band_task_list = [
        tasks.simple_glacier_masks,
        tasks.elevation_band_flowline,
        tasks.fixed_dx_elevation_band_flowline,
        tasks.compute_downstream_line,
        tasks.compute_downstream_bedshape,
    ]

    for task in elevation_band_task_list:
        workflow.execute_entity_task(task, gdirs);

elif flowline_type_to_use == 'centerline':
    # for centerline we can use parabola downstream line
    cfg.PARAMS['downstream_line_shape'] = 'parabola'

    centerline_task_list = [
        tasks.glacier_masks,
        tasks.compute_centerlines,
        tasks.initialize_flowlines,
        tasks.catchment_area,
        tasks.catchment_intersections,
        tasks.catchment_width_geom,
        tasks.catchment_width_correction,
        tasks.compute_downstream_line,
        tasks.compute_downstream_bedshape,
    ]

    for task in centerline_task_list:
        workflow.execute_entity_task(task, gdirs);
    
else:
    raise ValueError(f"Unknown flowline type '{flowline_type_to_use}'! Select 'elevation_band' or 'centerline'!")


In [ ]:
# define the climate data to use, we keep the default
cfg.PARAMS['baseline_climate'] = cfg.PARAMS['baseline_climate']

# add climate data to gdir
workflow.execute_entity_task(tasks.process_climate_data, gdirs);

# the default mb calibration
workflow.execute_entity_task(tasks.mb_calibration_from_geodetic_mb,
                             gdirs,
                             informed_threestep=True,  # only available for 'GSWP3_W5E5'
                            );

# glacier bed inversion
workflow.execute_entity_task(tasks.apparent_mb_from_any_mb, gdirs);
workflow.calibrate_inversion_from_consensus(
    gdirs,
    apply_fs_on_mismatch=True,
    error_on_mismatch=True,  # if you running many glaciers some might not work
    filter_inversion_output=True,  # this partly filters the overdeepening due to
    # the equilibrium assumption for retreating glaciers (see. Figure 5 of Maussion et al. 2019)
    volume_m3_reference=None,  # here you could provide your own total volume estimate in m3
);

# finally create the dynamic flowlines
workflow.execute_entity_task(tasks.init_present_time_glacier, gdirs);


In [ ]:
cfg.PARAMS['store_fl_diagnostics'] = True

# set the ice dynamic solver depending on the flowline-type
if flowline_type_to_use == 'elevation_band':
    cfg.PARAMS['evolution_model'] = 'SemiImplicit'
elif flowline_type_to_use == 'centerline':
    cfg.PARAMS['evolution_model'] = 'FluxBased'
else:
    raise ValueError(f"Unknown flowline type '{flowline_type_to_use}'! Select 'elevation_band' or 'centerline'!")

# get the start and end year of the selected baseline
y0 = gdirs[0].get_climate_info()['baseline_yr_0']
ye = gdirs[0].get_climate_info()['baseline_yr_1'] + 1  # run really to the end until 1.1.


# 'dynamic' initialisation, including dynamic mb calibration
dynamic_spinup_start_year = 1979
minimise_for = 'area'  # other option would be 'volume'
workflow.execute_entity_task(
    tasks.run_dynamic_melt_f_calibration, gdirs,
    err_dmdtda_scaling_factor=0.2,  # by default we reduce the mass balance error for accounting for
    # corrleated uncertainties on a regional scale
    ys=dynamic_spinup_start_year, ye=ye,
    kwargs_run_function={'minimise_for': minimise_for},
    ignore_errors=True,
    kwargs_fallback_function={'minimise_for': minimise_for},
    output_filesuffix='_spinup_historical',
    
);

## Selecting glaciers and setting their plotting/animation attributes here

currently selected a few reference glaciers from [this list](https://wgms.ch/products_ref_glaciers/) for different regions.  
Furthermore certain glaciers of own interest were added to the list(e.g. Upsala, Khumbu)

The glacier volume and area changes before that date are highly uncertain and serve the purpose of spinup only! In the ["10 minutes to... a dynamical spinup"](dynamical_spinup.ipynb) tutorial, we talk about why. For now, these files are perfect for our purpose, since we plan to start our simulation in 2020.

## Download and process GCM data from ISIMIP3b (bias-corrected CMIP6)

A typical use case for OGGM will be to use climate model output (here bias-corrected CMIP6 GCMs from [ISIMIP3b](https://www.isimip.org/gettingstarted/isimip3b-bias-adjustment/)). We use the files [we mirrored in Bremen](https://cluster.klima.uni-bremen.de/~oggm/cmip6/isimip3b/flat/monthly/) here, but you can use whichever you want. From ISIMIP3b, we have 5 GCMs and 3 SSPs on the cluster. You can find more information on the [ISIMIP website](https://www.isimip.org/gettingstarted/isimip3b-bias-adjustment). Let's download the data:

In [ ]:
# you can choose one of these 5 different GCMs:
# 'gfdl-esm4_r1i1p1f1', 'mpi-esm1-2-hr_r1i1p1f1', 'mri-esm2-0_r1i1p1f1' ("low sensitivity" models, within typical ranges from AR6)
# 'ipsl-cm6a-lr_r1i1p1f1', 'ukesm1-0-ll_r1i1p1f2' ("hotter" models, especially ukesm1-0-ll)
member = 'mri-esm2-0_r1i1p1f1' 

for ssp in ['ssp126', 'ssp370','ssp585']:
    # bias correct them
    workflow.execute_entity_task(gcm_climate.process_monthly_isimip_data, gdirs, 
                                 ssp = ssp,
                                 # gcm member -> you can choose another one
                                 member=member,
                                 # recognize the climate file for later
                                 output_filesuffix=f'_ISIMIP3b_{member}_{ssp}',
                                 );

The advantage of using ISIMIP3b data is that they have been bias-corrected by the ISIMIP consortium. Since we are using the [W5E5](https://docs.oggm.org/en/latest/climate-data.html#w5e5) dataset as the baseline climate in OGGM v1.6, there is no need for us to bias correct any further. If you want to bias-correct the projections yourself or want to have a larger variety of GCMs, you can also use the original CMIP5 or CMIP6 GCMs.

If you want to know which historical data you are using, you can ask OGGM:

In [ ]:
gdirs[0].get_climate_info()

## Projection runs 

We now run OGGM under various scenarios **starting from the end year of the historical spin-up run**:

In [ ]:
cfg.PARAMS['store_model_geometry'] = True
cfg.PARAMS['store_fl_diagnostics'] = True

In [ ]:
for ssp in ['ssp126', 'ssp370', 'ssp585']:
    rid = f'_ISIMIP3b_{member}_{ssp}'
    workflow.execute_entity_task(tasks.run_from_climate_data, gdirs,
                                 climate_filename='gcm_data',  # use gcm_data, not climate_historical
                                 climate_input_filesuffix=rid,  # use the chosen scenario
                                 init_model_filesuffix='_spinup_historical',  # this is important! Start from 2020 glacier
                                 output_filesuffix=rid,  # recognize the run for later
                                );

In [ ]:
from matplotlib.colors import ListedColormap
# simple cmap that is all white
cmap = ListedColormap(['white', 'white', 'white', 'white'])

add_mesh_ice_thick_args_new = {
    'cmap' : cmap,  # use a different matplotlib cmap
    'show_scalar_bar': False
    
}

In the following cell for each glacier and each of the 3 ssp scenarios the model data is distributed from 1D(flowline) to 2D(gridded) data and plotted with the [oggm-3dviz library](https://github.com/OGGM/oggm-3dviz).

In [ ]:
from oggm.sandbox import distribute_2d
from oggm_3dviz.tools import Glacier3DViz
from oggm_3dviz.tools.map_annotations import TextAnnotation

# animations path
animations_path = join(cfg.PATHS['working_dir'], 'animations')
if not os.path.exists(animations_path):
    os.makedirs(animations_path)
dist_ds_outs = []
for i, (glac_name, gdir) in enumerate(zip(glac_names, gdirs)):
    # This is to add a new topography to the file (smoothed differently)
    distribute_2d.add_smoothed_glacier_topo(gdir)
    # This is to get the bed map at the start of the simulation
    tasks.distribute_thickness_per_altitude(gdir)
    # This is to prepare the glacier directory for the interpolation (needs to be done only once)
    distribute_2d.assign_points_to_band(gdir)
    for ssp in ['ssp126', 'ssp370', 'ssp585']:
        rid = f'_ISIMIP3b_{member}_{ssp}'
        dist_ds, dist_ds_out = distribute_2d.distribute_thickness_from_simulation(gdir, 
                                                              input_filesuffix=rid, 
                                                              concat_input_filesuffix='_spinup_historical',
                                                              fl_thickness_threshold=glac_dict[glac_name]['fl_thickness_threshold'],
                                                                rolling_mean_smoothing=glac_dict[glac_name]['rolling_mean_smoothing'],
                                                                debug_area_timeseries=True,
                                                                add_monthly=True
                                                          )
        # saving area debugging data frames
        dist_ds_out['glac_name'] = glac_name
        dist_ds_out['scenario'] = ssp
        dist_ds_outs.append(dist_ds_out)
        
        with xr.open_dataset(gdir.get_filepath('gridded_simulation', filesuffix=f'_ISIMIP3b_{member}_{ssp}')) as ds:
            ds_glacier = ds.sel(time=slice(2000,2101))

            heading = TextAnnotation(
                text=f'{glac_name},\n scenario: {ssp}',
                position='upper_edge',
                font_size=16.5
                )

            if 'crop' in glac_dict[glac_name]:
                x_crop = glac_dict[glac_name]['crop']['x_crop']
                y_crop = glac_dict[glac_name]['crop']['y_crop']
            else:
                x_crop, y_crop = None, None
            print(glac_name, ds_glacier['topo'].shape)
            viz = Glacier3DViz(
                ds_glacier,  # dataset for visualization
                x='x',  # x-coordinate name, default is 'x'
                y='y',  # y-coordinate name, default is 'y'
                time='time',  # time coordinate name, default is 'time'
                topo_bedrock='topo',
                # topo_bedrock='bedrock',  # bedrock data variable, default is 'bedrock'
                use_texture=True, # use satellite texture for the topography
                ice_thickness='simulated_thickness',  # evolving ice thickness variable, default is 'simulated_thickness'
                x_crop=x_crop,
                y_crop=y_crop,
                time_var_display='calendar_year',# time format displayed, opting for total years here
                camera_args=glac_dict[glac_name]['camera_params'],
                additional_annotations=[heading],
                # add_mesh_ice_thick_args=add_mesh_ice_thick_args_new
                )
            viz.export_animation(filename=join(animations_path, f'{glac_name}_{ssp}_hi-res.mp4'),  # filename for the animation, should end with .mp4
                     framerate=80,  # framerate of the animation, affects duration and speed
                     quality=10,  # The range is 0 - 10. Higher quality leads to a larger file.
                    )
                
        

the following cell creates comparison plots for each glacier which has been modeled/animated. This shows the actually modeled area compared to the area which is used when plotting. This area adaption is done to smooth the animation and get rid of flickering.  
These plots are useful to check how much the actual model output is altered for animation purposes.

In [ ]:
# creating area plots
for dist in dist_ds_outs:
    glacier_name = dist['glac_name'].iloc[0]
    scenario = dist['scenario'].iloc[0]
    filename=join(animations_path, f'area_graphs_{glacier_name}_{scenario}.png')

    # Plot 'initial_area' and 'smoothed_area'
    plt.figure(figsize=(10, 6))
    plt.plot(dist['initial_area'], label='Initial Area')
    plt.plot(dist['smoothed_area'], label='Smoothed Area')
    plt.xlabel('Year')
    plt.ylabel('Area')
    plt.title(f'Glacier: {glacier_name}, Scenario: {scenario}')
    plt.legend()
    plt.savefig(filename)
    plt.close()